# Import Libraries

In [1]:
import random
import time
from collections import defaultdict
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets
from IPython.display import display, clear_output

In [2]:
# Node class represents an element in a doubly linked list
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [3]:
# DoublyLinkedList provides the base structure for Stack and Queue
class DoublyLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def insert(self, data):
        """Inserts a new node at the end of the list. Update head and tail as necessary"""
        try:
            new_node = Node(data)
            if self.head is None:
                self.head = self.tail = new_node
            else:
                new_node.prev = self.tail
                self.tail.next = new_node
                self.tail = new_node
        except Exception as e:
            print(f"Error inserting data: {e}")

    def remove(self, data):
        """Removes the first occurrence of a node with matching data. Updates prev and next pointers to maintain list's structure"""
        try:
            current = self.head
            while current:
                if current.data == data:
                    if current.prev:
                        current.prev.next = current.next
                    if current.next:
                        current.next.prev = current.prev
                    if current == self.head:
                        self.head = current.next
                    if current == self.tail:
                        self.tail = current.prev
                    return
                current = current.next
        except Exception as e:
            print(f"Error removing data: {e}")

    def to_list(self):
        """Converts the linked list to a Python list for easy display."""
        result = []
        current = self.head
        while current:
            result.append(current.data)
            current = current.next
        return result

In [4]:
# Stack implementation using DoublyLinkedList
# Purpose is to implement LIFO structure of stack using DoublyLinkedList
class Stack:
    def __init__(self):
        self.dll = DoublyLinkedList()
    # Add new element with respect to LIFO
    def insert(self, data):
        """Insert an element in LIFO order."""
        self.dll.insert(data)
    # Remove specific element from stack
    def remove(self, data):
        """Remove a specific element."""
        self.dll.remove(data)
    # Display stack as a list in LIFO order by reversing the linked list
    def display(self):
        """Display stack elements from top to bottom."""
        elements = self.dll.to_list()
        elements.reverse()
        return elements


# Queue implementation using DoublyLinkedList
# Purpose is to implement FIFO structure of queue using DoublyLinkedList
class Queue:
    def __init__(self):
        self.dll = DoublyLinkedList()
    # Add a new element with respect to FIFO
    def insert(self, data):
        """Insert an element in FIFO order."""
        self.dll.insert(data)
    # Remove a specific element from the queue
    def remove(self, data):
        """Remove a specific element."""
        self.dll.remove(data)
    # Display the queue as a list in FIFO order by converting the linked list 
    def display(self):
        """Display queue elements from front to rear."""
        return self.dll.to_list()

In [5]:
# Measures performance of Insert/Delete operations and visualizes results
def performance_test():
    sizes = [100, 1000, 10000, 100000, 1000000, 10000000]  # Larger dataset sizes
    stack = Stack() # Instantiate Stack
    queue = Queue() # Instantiate Queue
    results = {"Stack": defaultdict(list), "Queue": defaultdict(list)} # Store results of lists to defaultdict

    # Generate random data
    for size in sizes:
        data = [random.randint(1, 10000) for _ in range(size)]

        # Measure Insert performance for Stack
        start_time = time.time()
        for num in data:
            stack.insert(num)
        results["Stack"]["Insert"].append(time.time() - start_time)

        # Measure Delete performance for Stack
        start_time = time.time()
        for num in data:
            stack.remove(num)
        results["Stack"]["Delete"].append(time.time() - start_time)

        # Measure Insert performance for Queue
        start_time = time.time()
        for num in data:
            queue.insert(num)
        results["Queue"]["Insert"].append(time.time() - start_time)

        # Measure Delete performance for Queue
        start_time = time.time()
        for num in data:
            queue.remove(num)
        results["Queue"]["Delete"].append(time.time() - start_time)

    # Save results to Excel
    df_stack = pd.DataFrame(results["Stack"], index=sizes)
    df_queue = pd.DataFrame(results["Queue"], index=sizes)
    with pd.ExcelWriter("performance_results.xlsx") as writer:
        df_stack.to_excel(writer, sheet_name="Stack")
        df_queue.to_excel(writer, sheet_name="Queue")

    # Plot performance results using Plotly with logarithmic scaling
    fig = go.Figure()

    for op in ["Insert", "Delete"]:
        fig.add_trace(go.Scatter(
            name=f"Stack {op}",
            x=sizes,
            y=results["Stack"][op],
            mode="lines+markers",
            line=dict(dash="solid")
        ))
        fig.add_trace(go.Scatter(
            name=f"Queue {op}",
            x=sizes,
            y=results["Queue"][op],
            mode="lines+markers",
            line=dict(dash="dash")
        ))

    fig.update_layout(
        title="Performance Testing Results (Logarithmic Scale)",
        xaxis=dict(
            title="Dataset Size",
            type="log",
            tickvals=sizes,
            ticktext=[str(size) for size in sizes]
        ),
        yaxis=dict(
            title="Time (seconds)",
            type="log"
        ),
        legend_title="Operation and Data Structure",
        template="plotly_white"
    )

    fig.show()

In [6]:
# Provides an interactive demonstration of Stack and Queue operations
def visual_demo():
    stack = Stack() # Instantiate Stack
    queue = Queue() # Instantiate Queue

    def update(action, data_structure, value):
        """Updates the Stack or Queue interactively based on user inputs."""
        clear_output(wait=True)
        try:
            value = int(value)
        except ValueError:
            print("Invalid input! Please enter a valid integer.")
            return

        if data_structure == "Stack":
            if action == "Insert":
                stack.insert(value)
            elif action == "Remove":
                stack.remove(value)
            print("Stack (Top to Bottom):", stack.display())
        elif data_structure == "Queue":
            if action == "Insert":
                queue.insert(value)
            elif action == "Remove":
                queue.remove(value)
            print("Queue (Front to Rear):", queue.display())

    interact(
        update,
        action=["Insert", "Remove"],
        data_structure=["Stack", "Queue"],
        value=widgets.Text(value="", placeholder="Enter a number"),
    )

In [10]:
# Main function to run the script
if __name__ == "__main__":
    print("1. Performance Testing")
    print("2. Visual Demonstration")
    choice = input("Enter your choice (1 or 2): ")
    if choice == "1":
        performance_test()
    elif choice == "2":
        visual_demo()
    else:
        print("Invalid choice! Exiting.")

1. Performance Testing
2. Visual Demonstration


Enter your choice (1 or 2):  2


interactive(children=(Dropdown(description='action', options=('Insert', 'Remove'), value='Insert'), Dropdown(d…